## Extracted Action Models: raw vs filtered

The extraction of valid action models from VerbNet annotations is a multi-step process. First, VerbNet annotations is converted into a raw STRIPS format. Following this conversion, a subsequent processing stage is reuiqred to refine thses raw actions into a valid collection of action models. This section leverages this two-stage methodology to analyse the quality and suitability of VerbNet annotation for the task of action model extraction.

In [1]:
import json 

raw_STRIPS = '../examples/extracted_unfiltered_STRIPS.json'
# raw_STRIPS = '../output/action_model.json'
filtered_STRIPS = '../examples/extracted_filtered_STRIPS.json'

with open(raw_STRIPS, 'r') as f:
    raw_data = json.load(f)

with open(filtered_STRIPS, 'r') as f:
    filtered_data = json.load(f)

### Valid models in raw data

Theoretically, each frame schema within VerbNet annotaion can be mapped to an action model. However, numerous special cases prevent a one-to-one conversion. For instance, some frames may lack of valid predicates or fail to annotate the effects of an action. 

Defining a valid action model as one requiring at least one specificed effect, we can access the quality of the raw extraction. Based on the counting results, out of 1591 action frames initially extracted, only 1166 were found to satisfy the condition of having at least one valid effect and were therefore considered valid models.

In [2]:
# how many valid action models in raw
valid_count = 0
for entry in raw_data:
    frames = entry.get('frames', [])
    for frame in frames:
        postconditions = frame.get('postconditions', [])
        if len(postconditions) > 0:
             valid_count += 1

print(f"total action frames: {sum(len(entry.get('frames', [])) for entry in raw_data)}")
print(f"valid extracted action models: {valid_count} from {len(raw_data)} classes")

total action frames: 1591
valid extracted action models: 1166 from 602 classes


Furthermore, the following code analyses the invalid frames and reveals the distribution of failures across different VerbNet classes.

In 18 VerbNet classes, no valid action models could be extracted at all. A more common issue was observed in an additional 170 classes, where the extracted action models consistently lacked any annotated effects, thereby failing the validity criteria.

In [3]:
classes_with_no_postconditions = []
classes_with_no_valid_frame = []

for entry in raw_data:
    verb = entry.get('class_id', 'null')
    frames = entry.get('frames', [])
    if len(frames) == 0:
        classes_with_no_valid_frame.append(verb)
        continue
    if all(len(frame['postconditions']) == 0 for frame in frames):
        classes_with_no_postconditions.append(verb)

print(f"classes with no valid action model: {len(classes_with_no_valid_frame)}")
print(f"classes with no postconditions: {len(classes_with_no_postconditions)}")

classes with no valid action model: 18
classes with no postconditions: 170


### Valid model in filtered data

To ensure the consistency and integrity of the final dataset, a subsequent filetering stage is applied to remove both invalid and duplicated entrires from the set of valid models. This refinement process yields a final corpus of 432 unique action models, derived from 329 top-level classes.

In [4]:
print(f"action models after merging duplicates: {sum(len(entry.get('frames', [])) for entry in filtered_data)}")
print(f"classes after merging subclasses: {len(filtered_data)}")

action models after merging duplicates: 432
classes after merging subclasses: 329


The contribution of frames per class ranges widely. At the upper end, four classes (such as *break*) each provide a maximum of six distinct frames. In contracst, 87 classes fail to produce any valid frames.

In [5]:
# Count all classes with the most unique frames
max_frame_count = 0
min_frame_count = 0

for entry in filtered_data:
    frames = entry.get('frames', [])
    if len(frames) > max_frame_count:
        max_frame_count = len(frames)
    elif len(frames) < min_frame_count:
        min_frame_count = len(frames)

classes_w_max_frames = []
classes_w_min_frames = []

for entry in filtered_data:
    class_name = entry.get('class_id')
    frames = entry.get('frames', [])
    if len(frames) == max_frame_count:
        classes_w_max_frames.append(class_name)
    elif len(frames) == min_frame_count:
        classes_w_min_frames.append(class_name)

print(f"Most unique frames in a class: {max_frame_count}")
print(f"  class with most unique frames: {classes_w_max_frames[:1]}")
print(f"  classes with the most unique frames: {len(classes_w_max_frames)}")
print(f"Least unique frame in a class: {min_frame_count}")
print(f"  class with least unique frames: {classes_w_min_frames[:1]}")
print(f"  classes with the least unique frames: {len(classes_w_min_frames)}")

Most unique frames in a class: 6
  class with most unique frames: ['break']
  classes with the most unique frames: 4
Least unique frame in a class: 0
  class with least unique frames: ['act']
  classes with the least unique frames: 87


A similar analysis is performed on the filtered data to count the number of classes that lack of valid models. The results shows that the filetering process ultimately leaves 87 classes with no valid models, while the remaining each contain has at least one model that satisfies the basic requirements.

In [6]:
classes_with_no_postconditions = []
classes_with_no_valid_frame = []

for entry in filtered_data:
    verb = entry.get('class_id', 'null')
    frames = entry.get('frames', [])
    if len(frames) == 0:
        classes_with_no_valid_frame.append(verb)
        continue
    if all(len(frame['postconditions']) == 0 for frame in frames):
        classes_with_no_postconditions.append(verb)

print(f"classes with no valid action model: {len(classes_with_no_valid_frame)}")
print(f"classes with no postconditions: {len(classes_with_no_postconditions)}")

classes with no valid action model: 87
classes with no postconditions: 0
